<a href="https://colab.research.google.com/github/SandeshGitHub2077/Colab-Notebooks/blob/main/xlnet-project/xlnet_file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Get OKR train-test dataset from [here](https://github.com/SandeshGitHub2077/Colab-Notebooks/tree/main/xlnet-project/datasets) 

### Importing Libraries and data

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv
Saving train.csv to train.csv


In [ ]:
df_train = pd.read_csv("train.csv", encoding= 'unicode_escape')
df_test = pd.read_csv("test.csv", encoding= 'unicode_escape')

In [ ]:
df_train = df_train.dropna(axis=1)

In [ ]:
df_train.head()

,keyresult,target
0,Exceed Net Promoter Score (NPS) of over 8.0,1
1,Get 1000 survey responses to annual satisfact...,1
2,Conduct 50 phone interviews with top customers,1
3,Conduct 15 phone interviews with recently chu...,1
4,Present an action plan of 10 improvements for...,1


In [ ]:
df_train.target.value_counts()

1    537
0    304
Name: target, dtype: int64

In [ ]:
df_train.sample(10)['keyresult'].tolist()

['give an exclusive pre-launch update to customers and partners',
 'maintain expense to revenue ratio',
 'roll out a continuous 2-way feedback loop via 7 days surveys',
 'achieve a 7 days employee satisfaction / pulse score of 8+',
 ' Organize an online survey with min 100 answers to rate our various content\xa0',
 'onboard 10 new resellers that focus on the central region',
 'interview 50 prospective customers and get their initial feedback',
 'increase profit by 20% within next 2 365 dayss',
 'get free gym membership',
 'ensure every manager is doing regular 1-on-1 meetings with 2-way feedback']

### Cleaning
 - Remove unnecessary grammar

In [ ]:
! pip install tweet-preprocessor
import preprocessor as p
import re
# def clean_text(text):
#   text = text.replace("#","")
#   return p.clean(text)

def clean_text(text):
    text = re.sub(r"@[A-Za-z0-9]+", ' ', text)
    text = re.sub(r"https?://[A-Za-z0-9./]+", ' ', text)
    text = re.sub(r"[^a-zA-z.!?'0-9]", ' ', text)
    text = re.sub('\t', ' ',  text)
    text = re.sub(r" +", ' ', text)
    return text

In [ ]:
from tqdm.notebook import tqdm
tqdm.pandas()

df_train['clean_text'] = df_train['keyresult'].astype(str).progress_map(clean_text)
df_test['clean_text'] = df_test['keyresult'].astype(str).progress_map(clean_text)

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [ ]:
# splitting the data into training and test dataset
X = df_train['clean_text']
y = df_train['target']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
train_df = pd.DataFrame(X_train)
train_df['target'] = y_train

eval_df = pd.DataFrame(X_test)
eval_df['target'] = y_test

In [ ]:
train_df.shape, eval_df.shape

((672, 2), (169, 2))

In [ ]:
# transformers - SOTA implementation of pretrained models
!pip install -U simpletransformers 

     |████████████████████████████████| 225kB 8.5MB/s 
     |████████████████████████████████| 266kB 50.0MB/s 
     |████████████████████████████████| 1.8MB 41.3MB/s 
     |████████████████████████████████| 1.2MB 40.9MB/s 
     |████████████████████████████████| 3.3MB 40.2MB/s 
     |████████████████████████████████| 51kB 9.9MB/s 
     |████████████████████████████████| 2.5MB 25.2MB/s 
     |████████████████████████████████| 7.8MB 43.7MB/s 
     |████████████████████████████████| 133kB 55.9MB/s 
     |████████████████████████████████| 81kB 13.5MB/s 
     |████████████████████████████████| 122kB 57.5MB/s 
     |████████████████████████████████| 245kB 53.0MB/s 
     |████████████████████████████████| 174kB 56.6MB/s 
     |████████████████████████████████| 102kB 16.5MB/s 
     |████████████████████████████████| 143kB 56.9MB/s 
     |████████████████████████████████| 901kB 44.5MB/s 
     |████████████████████████████████| 112kB 56.6MB/s 
     |████████████████████████████████| 4.2MB 39.7MB

In [ ]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging
import sklearn


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

args = {
   'output_dir': 'outputs/',
   'cache_dir': 'cache/',
   'fp16': True,
   'fp16_opt_level': 'O1',
   'max_seq_length': 128,
   'train_batch_size': 32,
   'eval_batch_size': 8,
   'gradient_accumulation_steps': 1,
   'num_train_epochs': 6,
   'weight_decay': 0,
   'learning_rate': 4e-5,
   'adam_epsilon': 1e-8,
   'warmup_ratio': 0.06,
   'warmup_steps': 0,
   'max_grad_norm': 1.0,
   'logging_steps': 50,
   'evaluate_during_training': False,
   'save_steps': 2000,
   'eval_all_checkpoints': True,
   'use_tensorboard': True,
   'overwrite_output_dir': True,
   'reprocess_input_data': False,
}



# ClassificationModel
model = ClassificationModel('xlnet', 'xlnet-base-cased', args={'num_train_epochs':6, 'train_batch_size':32, 'max_seq_length':128})

# Model
model.train_model(train_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df, acc=sklearn.metrics.accuracy_score)

INFO:filelock:Lock 139798568291792 acquired on /root/.cache/huggingface/transformers/06bdb0f5882dbb833618c81c3b4c996a0c79422fa2c95ffea3827f92fc2dba6b.da982e2e596ec73828dbae86525a1870e513bd63aae5a2dc773ccc840ac5c346.lock


INFO:filelock:Lock 139798568291792 released on /root/.cache/huggingface/transformers/06bdb0f5882dbb833618c81c3b4c996a0c79422fa2c95ffea3827f92fc2dba6b.da982e2e596ec73828dbae86525a1870e513bd63aae5a2dc773ccc840ac5c346.lock


INFO:filelock:Lock 139798568000272 acquired on /root/.cache/huggingface/transformers/9461853998373b0b2f8ef8011a13b62a2c5f540b2c535ef3ea46ed8a062b16a9.3e214f11a50e9e03eb47535b58522fc3cc11ac67c120a9450f6276de151af987.lock


INFO:filelock:Lock 139798568000272 released on /root/.cache/huggingface/transformers/9461853998373b0b2f8ef8011a13b62a2c5f540b2c535ef3ea46ed8a062b16a9.3e214f11a50e9e03eb47535b58522fc3cc11ac67c120a9450f6276de151af987.lock


Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.bias', 'logits_proj.bias', 'sequence_summary.summary.weight', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

...

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_xlnet_128_2_2


/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:936: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  model.parameters(), args.max_grad_norm
/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarnin

INFO:simpletransformers.classification.classification_model: Training of xlnet model complete. Saved to outputs/.
/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1393: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_xlnet_128_2_2


INFO:simpletransformers.classification.classification_model:{'mcc': 0.9486945962355798, 'tp': 106, 'tn': 59, 'fp': 2, 'fn': 2, 'auroc': 0.9892228293867638, 'auprc': 0.9956225886246401, 'acc': 0.9763313609467456, 'eval_loss': 0.10220809348604896}


In [ ]:
result

{'acc': 0.9763313609467456,
 'auprc': 0.9956225886246401,
 'auroc': 0.9892228293867638,
 'eval_loss': 0.10220809348604896,
 'fn': 2,
 'fp': 2,
 'mcc': 0.9486945962355798,
 'tn': 59,
 'tp': 106}

In [ ]:
predictions, raw_outputs = model.predict(df_test.clean_text.tolist())

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


### Saving model on a local file to skip the execution time next time we predict

In [ ]:
import pickle
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))

Loading model

In [ ]:
loaded_model = pickle.load(open(filename, 'rb'))

### Lets Put the Input user and its processing in a function

In [ ]:
def singleInput():
  userText = input("Enter KR:\n")
  userText = clean_text(userText)
  predictions1 = loaded_model.predict(userText)
  print('Measurable' if predictions1[0][0] == 1 else 'Non-Measurable')

In [ ]:
singleInput()

Enter KR:
well hello my name is lo9ca


INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.



Non-Measurable
